# Models

This notebook implements the full experimental pipeline described in the manuscript:

**“A rigorous statistical benchmark of global neural time-series models for multi-horizon air pollutant forecasting.”**

Its purpose is to:

- Define and train the fifteen forecasting models evaluated in the study;
- Execute rolling-origin cross-validation (ROCV) across the four forecast horizons (1, 7, 14, and 30 days);
- Generate multi-step forecasts for each pollutant and grid cell;
- Compute evaluation metrics (MAE, RMSE, sMAPE);
- Apply deterministic scaling for numerical stability in neural training;

This notebook reproduces all numerical results used in the performance and inferential analyses of the manuscript.

## Libraries

In [ ]:
import os
os.environ['NIXTLA_ID_AS_COL'] = '1'
import glob

import optuna
import itertools
import shutil
import time
import functools
import gc

import pandas as pd
import numpy as np
np.random.seed(1)
import scipy.stats as stats

import plotly.graph_objects as go
import plotly.express as px
import plotly.subplots
import plotly.io as pio
from graphmodex import plotlymodex
pio.renderers.default = 'notebook'
from plotly.subplots import make_subplots

import joblib
import pickle
from IPython.display import clear_output

c:\Users\gustavo.filho\Documents\Python\Masters\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import neuralforecast
import mlforecast
import statsforecast
import utilsforecast
import coreforecast

from statsforecast import StatsForecast
from statsforecast.models import (
    Naive, SeasonalNaive, 
    AutoARIMA, AutoCES, AutoETS, AutoTheta,
)

from mlforecast import MLForecast
import lightgbm as lgb
from sklearn.linear_model import LinearRegression
from mlforecast.lag_transforms import ExpandingMean, RollingMean
from mlforecast.target_transforms import Differences
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor

from neuralforecast import NeuralForecast
from neuralforecast.models import (
    NBEATS, NHITS,
    GRU, Informer, LSTM
)
from neuralforecast.losses.pytorch import MSE, SMAPE, MAE

from mlforecast.utils import PredictionIntervals

from pytorch_lightning import Trainer
trainer = Trainer(
    max_steps=4,
    logger=False,
    enable_progress_bar=False,
    enable_model_summary=False
)

import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="optuna")


# ==================================================
# REPRODUCTIBILITY
# ==================================================
import random
import torch

SEED = 1
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

2026-02-16 08:45:46,189	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2026-02-16 08:45:46,654	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
💡 Tip: For seamless cloud logging and experiment tracking, try installing [litlogger](https://pypi.org/project/litlogger/) to enable LitLogger, which logs metrics and artifacts automatically to the Lightning Experiments platform.


In [3]:
import warnings
warnings.filterwarnings(
    "ignore",
    message="overflow encountered in square",
    category=RuntimeWarning,
)

In [4]:
def print_gpu_utilization():
    if torch.cuda.is_available():
        usage = torch.cuda.memory_reserved() / 1024**3 # GB
        total = torch.cuda.get_device_properties(0).total_memory / 1024**3
        print(f"GPU usage: {usage:.2f}GB / {total:.2f}GB")

# Call this at the start of your loop

### Results Storage

In [5]:
from pathlib import Path

BASE_RESULTS = Path("Results/RQ1")
BASE_RESULTS.mkdir(parents=True, exist_ok=True)

def save_results(df, model_family, pollutant, horizon_label):
    out_dir = BASE_RESULTS / model_family
    out_dir.mkdir(parents=True, exist_ok=True)

    fname = f"{pollutant}_{horizon_label}.csv"
    df.to_csv(out_dir / fname, index=False)

BASE_DIR_Pilot = Path("Results/RQ1/Pilot")
(BASE_DIR_Pilot / "studies").mkdir(parents=True, exist_ok=True)
(BASE_DIR_Pilot / "tables").mkdir(parents=True, exist_ok=True)
(BASE_DIR_Pilot / "meta").mkdir(parents=True, exist_ok=True)

## Data & Ablation

In [6]:
# ===============================
# DATA
# ===============================
df = pd.read_parquet(r'..\Data\CAMS\processed\eac4_era5_2010_2024_brasil_enhanced.parquet')

pm10 = (
    df
    .rename(columns={
        'pm10': 'y',
        'valid_time': 'ds'        
    })
    .query('state == "Paraná"')
    [['unique_id', 'ds', 'y']]
)

pm2p5 = (
    df
    .rename(columns={
        'pm2p5': 'y',
        'valid_time': 'ds'        
    })
    .query('state == "Paraná"')
    [['unique_id', 'ds', 'y']]
)

go3 = (
    df
    .rename(columns={
        'go3': 'y',
        'valid_time': 'ds'        
    })
    .query('state == "Paraná"')
    [['unique_id', 'ds', 'y']]
)

no2 = (
    df
    .rename(columns={
        'no2': 'y',
        'valid_time': 'ds'        
    })
    .query('state == "Paraná"')
    [['unique_id', 'ds', 'y']]
)

In [7]:
# go3.to_parquet(r'.\Results\RQ1\full\go3.parquet')
# pm10.to_parquet(r'.\Results\RQ1\full\nopm102.parquet')
# pm2p5.to_parquet(r'.\Results\RQ1\full\nopm2p52.parquet')
# no2.to_parquet(r'.\Results\RQ1\full\no2.parquet')

### Setup

In [8]:
# ===============================
# PILOT CONFIG
# ===============================
POLLUTANT_abl = 'pm10'
UID_abl = 0

steps_per_day = 8
horizons = {
    '1d': 1 * steps_per_day,
    '7d': 7 * steps_per_day,
    '14d': 14 * steps_per_day,
    '30d': 30 * steps_per_day,
}

# ===============================
# DATA
# ===============================
df_abl = (
    df
    .query("unique_id == @UID_abl")
    .rename(columns={'pm10': 'y', 'valid_time': 'ds'})
    [['unique_id', 'ds', 'y']]
    .sort_values('ds')
)

In [9]:
# ===============================
# CONFIG
# ===============================
steps_per_day = 8
two_year_steps = 2 * 365 * steps_per_day
target_windows = 30

FREQ = '3h'
SEASON_LENGTH = 8 

pollutants_dict = {
        'go3': {
            'df': go3,
            'scaler': 1e8
        },
        'no2': {
            'df': no2,
            'scaler': 1e10
        },
        'pm10': {
            'df': pm10,
            'scaler': 1e9
        },
        'pm2p5': {
            'df': pm2p5,
            'scaler': 1e9
        },
    }
experiments_dict = {
    '1 days': {
        'horizon': 8*1,
        'step_size': max(8*1, two_year_steps // target_windows),
        'windows': target_windows,
    },
    '7 days': {
        'horizon': 8*7,
        'step_size': max(8*7, two_year_steps // target_windows),
        'windows': target_windows,
    },
    '14 days': {
        'horizon': 8*14,
        'step_size': max(8*14, two_year_steps // target_windows),
        'windows': target_windows,
    },
    '30 days': {
        'horizon': 8*30,
        'step_size': 8*30,
        'windows': two_year_steps // (8*30),  # 24
    },
}

## **Models**

### Refit Ablation

In [ ]:
# =========================================================
# Refit Sensitivity Test Configuration
# =========================================================

TEST_POLLUTANT = list(pollutants_dict.keys())[0]        # Select first pollutant
TEST_HORIZON_LABEL = list(experiments_dict.keys())[2]   # Select third horizon
H = experiments_dict[TEST_HORIZON_LABEL]['horizon']
WINDOWS = 5                                             # Reduced for computational efficiency
STEP_SIZE = experiments_dict[TEST_HORIZON_LABEL]['step_size']

refit_comparison = []

# =========================================================
# Comparative Loop: refit=True vs refit=False
# =========================================================
for refit_mode in [True, False]:

    print(f"\nTesting configuration with refit={refit_mode}...")
    
    # Simplified model configuration for sensitivity analysis
    model = NHITS(
        h=H,
        input_size=14 * 8,
        max_steps=300,
        random_seed=SEED,
        loss=MAE()
    )
    
    nf = NeuralForecast(models=[model], freq=FREQ)

    df_test = pollutants_dict[TEST_POLLUTANT]['df'].copy()

    # Deterministic scaling for numerical stability (consistent with main experiment)
    df_test['y'] = df_test['y'] * 1e8

    # Time tracking
    start_t = time.time()
    
    cv_results = nf.cross_validation(
        df=df_test,
        h=H,
        n_windows=WINDOWS,
        step_size=STEP_SIZE,
        refit=refit_mode
    )

    # Rescale forecasts back to physical units
    cv_results['y'] = cv_results['y'] * 1e-8
    cv_results['NHITS'] = cv_results['NHITS'] * 1e-8
    
    end_t = time.time()
    duration = end_t - start_t
    
    # Compute global mean absolute error across CV windows
    mae_val = np.mean(np.abs(cv_results['y'] - cv_results['NHITS']))
    
    refit_comparison.append({
        'refit': refit_mode,
        'MAE': mae_val,
        'Time_Sec': duration,
        'Time_Per_Window': duration / WINDOWS
    })


# =========================================================
# Results Analysis
# =========================================================
comparison_df = pd.DataFrame(refit_comparison)

# Percentage change in MAE between configurations
comparison_df['MAE_Diff_Pct'] = comparison_df['MAE'].pct_change() * 100

# Relative speed-up factor
comparison_df['Speedup_Factor'] = (
    comparison_df.iloc[0]['Time_Sec'] / comparison_df['Time_Sec']
)

print("\n=== REFIT TRUE vs FALSE COMPARISON ===")
print(comparison_df.to_string(index=False))

# Logical consistency check
mae_diff = abs(
    comparison_df.iloc[0]['MAE'] - comparison_df.iloc[1]['MAE']
)

print(f"\nAbsolute MAE difference: {mae_diff:.6f}")

if mae_diff < 0.01:  # Threshold should be interpreted relative to scale
    print("Conclusion: The error difference is negligible. refit=False is acceptable.")
else:
    print("Warning: A noticeable error difference was detected. Verify stability assumptions.")

In [ ]:
# =========================================================
# Refit Sensitivity Test Configuration
# =========================================================

TEST_POLLUTANT = list(pollutants_dict.keys())[0]        # Select first pollutant
TEST_HORIZON_LABEL = list(experiments_dict.keys())[2]   # Select third horizon
H = experiments_dict[TEST_HORIZON_LABEL]['horizon']
WINDOWS = 5                                             # Reduced for computational efficiency
STEP_SIZE = experiments_dict[TEST_HORIZON_LABEL]['step_size']

refit_comparison = []

# =========================================================
# Comparative Loop: refit=True vs refit=False
# =========================================================
for refit_mode in [True, False]:

    print(f"\nTesting configuration with refit={refit_mode}...")
    
    # Simplified model configuration for sensitivity analysis
    model = NHITS(
        h=H,
        input_size=14 * 8,
        max_steps=300,
        random_seed=SEED,
        loss=MAE()
    )
    
    nf = NeuralForecast(models=[model], freq=FREQ)

    df_test = pollutants_dict[TEST_POLLUTANT]['df'].copy()

    # Deterministic scaling for numerical stability (consistent with main experiment)
    df_test['y'] = df_test['y'] * 1e8

    # Time tracking
    start_t = time.time()
    
    cv_results = nf.cross_validation(
        df=df_test,
        h=H,
        n_windows=WINDOWS,
        step_size=STEP_SIZE,
        refit=refit_mode
    )

    # Rescale forecasts back to physical units
    cv_results['y'] = cv_results['y'] * 1e-8
    cv_results['NHITS'] = cv_results['NHITS'] * 1e-8
    
    end_t = time.time()
    duration = end_t - start_t
    
    # Compute global mean absolute error across CV windows
    mae_val = np.mean(np.abs(cv_results['y'] - cv_results['NHITS']))
    
    refit_comparison.append({
        'refit': refit_mode,
        'MAE': mae_val,
        'Time_Sec': duration,
        'Time_Per_Window': duration / WINDOWS
    })


# =========================================================
# Results Analysis
# =========================================================
comparison_df = pd.DataFrame(refit_comparison)

# Percentage change in MAE between configurations
comparison_df['MAE_Diff_Pct'] = comparison_df['MAE'].pct_change() * 100

# Relative speed-up factor
comparison_df['Speedup_Factor'] = (
    comparison_df.iloc[0]['Time_Sec'] / comparison_df['Time_Sec']
)

print("\n=== REFIT TRUE vs FALSE COMPARISON ===")
print(comparison_df.to_string(index=False))

# Logical consistency check
mae_diff = abs(
    comparison_df.iloc[0]['MAE'] - comparison_df.iloc[1]['MAE']
)

print(f"\nAbsolute MAE difference: {mae_diff:.6f}")

if mae_diff < 0.01:  # Threshold should be interpreted relative to scale
    print("Conclusion: The error difference is negligible. refit=False is acceptable.")
else:
    print("Warning: A noticeable error difference was detected. Verify stability assumptions.")

### Input Size Ablation

In [ ]:
# =================================================================================
# DIEBOLD-MARIANO TEST (robust for multiple horizons) Just for Ablation
# =================================================================================

def diebold_mariano_test(e1, e2, h=1, power=1):
    """
    e1, e2: forecast error arrays (y - y_hat)
    h: forecast horizon
    power: 1 = MAE, 2 = MSE
    """

    d = np.abs(e1)**power - np.abs(e2)**power
    mean_d = np.mean(d)
    T = len(d)

    # HAC variance (simplified Newey-West style)
    gamma = []
    for lag in range(1, min(h, T)):
        cov = np.cov(d[:-lag], d[lag:])[0, 1]
        gamma.append(cov)

    var_d = np.var(d) + 2 * np.sum(gamma)

    dm_stat = mean_d / np.sqrt(var_d / T)
    p_value = 2 * (1 - stats.norm.cdf(np.abs(dm_stat)))

    return dm_stat, p_value


# =====================================================
# CONFIGURATIONS
# =====================================================

INPUT_SIZES = {
    "7d": 7 * 8,
    "14d": 14 * 8,
    "21d": 21 * 8
}

HORIZONS_TO_TEST = ["1 days", "30 days"]

MODELS = {
    "GRU": GRU,
    "LSTM": LSTM,
    "NBEATS": NBEATS,
    "NHITS": NHITS,
    "Informer": Informer,
}

all_predictions = []

# =====================================================
# MAIN LOOP — GENERATE FORECASTS
# =====================================================

for pollutant_name, pollutant_ in pollutants_dict.items():

    if pollutant_name != 'pm10':
        continue

    for horizon_label, experiment_ in experiments_dict.items():

        if horizon_label not in HORIZONS_TO_TEST:
            continue

        for model_name, ModelClass in MODELS.items():

            for input_label, input_size in INPUT_SIZES.items():

                print(f"{model_name} | {horizon_label} | {input_label}")

                model = ModelClass(
                    h=experiment_['horizon'],
                    input_size=input_size,
                    max_steps=200,
                    learning_rate=1e-3,
                    batch_size=32,
                    windows_batch_size=512,
                    random_seed=SEED,
                    alias=model_name,
                    logger=False,
                    loss=MAE(),
                )

                nf = NeuralForecast(models=[model], freq=FREQ)

                df_ = pollutant_['df'].copy()
                df_.y = df_.y * pollutant_['scaler']

                results_nf = nf.cross_validation(
                    df=df_,
                    h=experiment_['horizon'],
                    n_windows=experiment_['windows'],
                    step_size=experiment_['step_size'],
                    refit=False,
                )

                # scale back
                scale = pollutant_['scaler']
                results_nf['y'] /= scale
                results_nf[model_name] /= scale

                results_nf['model'] = model_name
                results_nf['input_label'] = input_label
                results_nf['horizon'] = horizon_label

                all_predictions.append(results_nf)

# Consolidate all forecasts
pred_df = pd.concat(all_predictions).reset_index(drop=True)


# =====================================================
# DIEBOLD-MARIANO BETWEEN INPUT_SIZES
# =====================================================

dm_results = []

for model_name in MODELS.keys():

    for horizon_label in HORIZONS_TO_TEST:

        subset = pred_df[
            (pred_df['model'] == model_name) &
            (pred_df['horizon'] == horizon_label)
        ]

        horizon_value = experiments_dict[horizon_label]['horizon']

        input_labels = list(INPUT_SIZES.keys())

        for i in range(len(input_labels)):
            for j in range(i + 1, len(input_labels)):

                input_a = input_labels[i]
                input_b = input_labels[j]

                df_a = subset[subset['input_label'] == input_a]
                df_b = subset[subset['input_label'] == input_b]

                e1 = df_a['y'].values - df_a[model_name].values
                e2 = df_b['y'].values - df_b[model_name].values

                dm_stat, p_value = diebold_mariano_test(
                    e1, e2,
                    h=horizon_value,
                    power=1
                )

                dm_results.append({
                    "model": model_name,
                    "horizon": horizon_label,
                    "input_a": input_a,
                    "input_b": input_b,
                    "dm_stat": dm_stat,
                    "p_value": p_value
                })

dm_df = pd.DataFrame(dm_results)

print("\nDiebold-Mariano between input_sizes:")
display(dm_df.sort_values(["model", "horizon", "input_a"]))

### Epochs Ablation

In [ ]:
from pytorch_lightning.loggers import CSVLogger

# =====================================================
# DATA CONFIGURATION
# =====================================================

pollutant_name = list(pollutants_dict.keys())[0]
pollutant_ = pollutants_dict[pollutant_name]

horizon = 56
input_size = 56
FREQ = "3H"
SEED = 42

df_ = pollutant_["df"].copy()
df_.y = df_.y * pollutant_["scaler"]

cutoff = df_["ds"].max() - pd.Timedelta(days=30)
train_df = df_[df_["ds"] <= cutoff]

# =====================================================
# MODEL DEFINITIONS
# =====================================================

model_configs = [
    (Informer, "Informer"),
    (GRU, "GRU"),
    (LSTM, "LSTM"),
    (NHITS, "NHITS"),
    (NBEATS, "NBEATS"),
]

log_dir = Path("results") / "rq1" / "logs_science"
log_dir.mkdir(parents=True, exist_ok=True)

models = []

for model_class, name in model_configs:

    logger = CSVLogger(str(log_dir), name=name)

    model = model_class(
        h=horizon,
        input_size=input_size,
        max_steps=1500,
        learning_rate=1e-3,
        batch_size=32,
        windows_batch_size=1024,
        random_seed=SEED,
        alias=name,
        loss=MAE(),
        logger=logger,
    )

    models.append(model)

# =====================================================
# TRAINING
# =====================================================

nf = NeuralForecast(models=models, freq=FREQ)
nf.fit(df=train_df, val_size=horizon)

print("Training of all models completed.")


# =====================================================
# LOAD METRICS
# =====================================================

def get_latest_metrics(model_alias):
    model_log_path = log_dir / model_alias
    versions = sorted(
        glob.glob(str(model_log_path / "version_*")),
        key=os.path.getmtime
    )
    if not versions:
        return None

    csv_path = Path(versions[-1]) / "metrics.csv"
    return pd.read_csv(csv_path).groupby("step").mean().reset_index()


# =====================================================
# PLOT LEARNING CURVES
# =====================================================

colors = {
    "NHITS": "#104266",
    "NBEATS": "#c49962",
    "Informer": "#6db16d",
    "LSTM": "#bb69b7",
    "GRU": "#9c3e3e",
}

fig = go.Figure()

for _, name in model_configs:
    metrics = get_latest_metrics(name)

    if metrics is not None:
        fig.add_trace(
            go.Scatter(
                x=metrics["step"],
                y=metrics["train_loss_epoch"],
                mode="lines",
                name=name,
                line=dict(color=colors[name], width=2),
                legendgroup=name,
            )
        )

fig.update_layout(
    template="simple_white",
    title={
        "text": f"Learning Curves Comparison: {pollutant_name.upper()}",
        "font": {"family": "Times New Roman", "size": 24},
        "y": 0.95,
        "x": 0.5,
        "xanchor": "center",
        "yanchor": "top",
    },
    xaxis=dict(
        title=dict(text="Training Steps", font=dict(family="Times New Roman", size=20)),
        tickfont=dict(family="Times New Roman", size=16),
        showline=True,
        linewidth=1.2,
        linecolor="black",
        mirror=True,
        showgrid=True,
        gridcolor="rgba(0,0,0,0.12)",
        zeroline=False,
    ),
    yaxis=dict(
        title=dict(text="Mean Absolute Error (MAE)", font=dict(family="Times New Roman", size=20)),
        tickfont=dict(family="Times New Roman", size=16),
        showline=True,
        linewidth=1.2,
        linecolor="black",
        mirror=True,
        showgrid=True,
        gridcolor="rgba(0,0,0,0.12)",
        zeroline=False,
    ),
    legend=dict(
        font=dict(family="Times New Roman", size=14),
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="center",
        x=0.48,
    ),
    width=900,
    height=600,
    plot_bgcolor="white",
    paper_bgcolor="white",
    margin=dict(l=80, r=40, t=100, b=80),
)

fig.show()

# =====================================================
# SAVE PUBLICATION-READY FIGURE
# =====================================================

output_dir = Path("results") / "rq1" / "figures"
output_dir.mkdir(parents=True, exist_ok=True)

output_path = output_dir / "learning_curves.pdf"

fig.write_image(
    str(output_path),
    format="pdf",
    width=700,
    height=600,
    scale=3,
)

print(f"Figure saved at: {output_path}")

### Deep Learners

In [ ]:
for pollutant_name, pollutant_ in pollutants_dict.items():
    for horizon_label, experiment_ in experiments_dict.items():
        try:
            print_gpu_utilization()
            clear_output(wait=True)
            print(f"Running NEURAL | {pollutant_name} | {horizon_label}")

            # -------------------------------
            # MODELS
            # -------------------------------
            models = [
                GRU(
                    h=experiment_['horizon'],
                    input_size=14 * 8,
                    max_steps=1000,  # Maximum training steps
                    learning_rate=1e-3,  # Learning rate
                    batch_size=32,
                    windows_batch_size=1024,
                    random_seed=SEED,
                    alias='GRU',  # Alias for identification
                    loss=MAE(),  # Loss function
                    logger=False,
                ),
                Informer(
                    h=experiment_['horizon'],
                    input_size=14 * 8,
                    max_steps=1500,  # Maximum training steps
                    learning_rate=1e-3,  # Learning rate
                    batch_size=32,
                    windows_batch_size=1024,
                    random_seed=SEED,
                    alias='Informer',
                    loss=MAE(),
                    logger=False,
                ),
                LSTM(
                    h=experiment_['horizon'],
                    input_size=14 * 8,
                    max_steps=1000,  # Maximum training steps
                    learning_rate=1e-3,  # Learning rate
                    batch_size=32,
                    windows_batch_size=1024,
                    random_seed=SEED,
                    alias='LSTM',
                    loss=MAE(),
                    logger=False,
                ),
            ]

            nf = NeuralForecast(models=models, freq=FREQ)

            # -------------------------------
            # DATA
            # -------------------------------
            df_ = pollutant_['df'].copy()
            df_.y = df_.y * pollutant_['scaler']

            start_time = time.time()
            results_nf = nf.cross_validation(
                df=df_,
                h=experiment_['horizon'],
                n_windows=experiment_['windows'],
                step_size=experiment_['step_size'],
                refit=False,
            )
            end_time = time.time()

            total_time = end_time - start_time
            time_per_window = total_time / experiment_['windows']
            results_nf['fit_time_seconds'] = total_time

            # -------------------------------
            # SCALE BACK
            # -------------------------------
            scale = pollutant_['scaler']
            for col in ['y', 'GRU', 'Informer', 'LSTM']:
                results_nf[col] = results_nf[col] / scale

            # -------------------------------
            # SAVE
            # -------------------------------
            save_results(
                results_nf,
                model_family="dl",
                pollutant=pollutant_name,
                horizon_label=horizon_label.replace(" ", "")
            )

            del nf
            del models
            gc.collect()
            torch.cuda.empty_cache()

        except Exception as e:
            print(f"Failed on {pollutant_name} {horizon_label}: {e}")
            continue  # Move to next instead of crashing the whole notebook

### Neural Forecasters

In [ ]:
for pollutant_name, pollutant_ in pollutants_dict.items():
    for horizon_label, experiment_ in experiments_dict.items():
        try:
            print_gpu_utilization()
            clear_output(wait=True)
            print(f"Running NEURAL | {pollutant_name} | {horizon_label}")

            # -------------------------------
            # MODELS
            # -------------------------------
            models = [
                NBEATS(
                    h=experiment_['horizon'],
                    input_size=14*8,
                    stack_types=["identity", "trend", "seasonality"],
                    n_blocks=[1, 1, 1],
                    mlp_units=3 * [[256, 256]],
                    basis='polynomial',
                    n_basis=2,
                    n_harmonics=2,
                    shared_weights=True,
                    activation='ReLU',
                    max_steps=1000,
                    learning_rate=1e-3,
                    batch_size=32,
                    windows_batch_size=1024,
                    random_seed=SEED,
                    alias='NBEATS-I',
                    loss=MAE(),
                ),
                NBEATS(
                    h=experiment_['horizon'],
                    input_size=14*8,
                    stack_types=['identity'] * 3,
                    n_blocks=[2, 2, 2],
                    mlp_units=3 * [[256, 256]],
                    shared_weights=False,
                    activation='ReLU',
                    max_steps=1000,
                    learning_rate=1e-3,
                    batch_size=32,
                    windows_batch_size=1024,
                    random_seed=SEED,
                    alias='NBEATS-G',
                    logger=False,
                    loss=MAE(),
                ),
                NHITS(
                    h=experiment_['horizon'],
                    input_size=14*8,
                    n_blocks=[1, 1, 1],
                    mlp_units=3 * [[256, 256]],
                    n_pool_kernel_size=[2, 2, 1],
                    n_freq_downsample=[4, 2, 1],
                    activation='ReLU',
                    dropout_prob_theta=0.1,
                    max_steps=1000,
                    learning_rate=1e-3,
                    batch_size=32,
                    windows_batch_size=1024,
                    random_seed=SEED,
                    alias='NHITS',
                    logger=False,
                    loss=MAE(),
                ),
            ]

            nf = NeuralForecast(models=models, freq=FREQ)

            # -------------------------------
            # DATA
            # -------------------------------
            df_ = pollutant_['df'].copy()
            df_.y = df_.y * pollutant_['scaler']

            start_time = time.time()
            results_nf = nf.cross_validation(
                df=df_,
                h=experiment_['horizon'],
                n_windows=experiment_['windows'],
                step_size=experiment_['step_size'],
                refit=False,
            )
            end_time = time.time()
            
            total_time = end_time - start_time
            time_per_window = total_time / experiment_['windows']
            results_nf['fit_time_seconds'] = total_time

            # -------------------------------
            # SCALE BACK
            # -------------------------------
            scale = pollutant_['scaler']
            for col in ['y', 'NHITS', 'NBEATS-G', 'NBEATS-I']:
                results_nf[col] = results_nf[col] / scale

            # -------------------------------
            # SAVE
            # -------------------------------
            save_results(
                results_nf,
                model_family="neural",
                pollutant=pollutant_name,
                horizon_label=horizon_label.replace(" ", "")
            )

            del nf
            del models
            gc.collect()
            torch.cuda.empty_cache()
            
        except Exception as e:
            print(f"Failed on {pollutant_name} {horizon_label}: {e}")
            continue  # Move to next instead of crashing the whole notebook

### Statistical Forecasters

In [ ]:
for pollutant_name, pollutant_ in pollutants_dict.items():
    for horizon_label, experiment_ in experiments_dict.items():

        print(f"Running STATS | {pollutant_name} | {horizon_label}")

        # -------------------------------
        # MODELS
        # -------------------------------
        stat_models = [
            AutoARIMA(season_length=SEASON_LENGTH, alias='Arima'),
            AutoETS(season_length=SEASON_LENGTH, alias='ETS'),
            AutoTheta(season_length=SEASON_LENGTH, alias='Theta'),
            SeasonalNaive(season_length=SEASON_LENGTH, alias='SeasonalNaive-8'),
            SeasonalNaive(season_length=56, alias='SeasonalNaive-56'),
            Naive(alias='Naive'),
        ]

        sf = StatsForecast(
            models=stat_models,
            freq=FREQ,
            n_jobs=1,
        )

        # -------------------------------
        # DATA
        # -------------------------------
        df_ = pollutant_['df'].copy()

        # -------------------------------
        # CROSS-VALIDATION
        # -------------------------------
        start_time = time.time()
        results_sf = sf.cross_validation(
            df=df_,
            h=experiment_['horizon'],
            n_windows=experiment_['windows'],
            step_size=experiment_['step_size'],
            input_size=14*8,
            refit=False,
        )
        end_time = time.time()
        
        total_time = end_time - start_time
        time_per_window = total_time / experiment_['windows']
        results_sf['fit_time_seconds'] = total_time
        
        # -------------------------------
        # SAVE
        # -------------------------------
        save_results(
            results_sf,
            model_family="stats",
            pollutant=pollutant_name,
            horizon_label=horizon_label.replace(" ", "")
        )

Running STATS | go3 | 1 days
Running STATS | go3 | 7 days
Running STATS | go3 | 14 days
Running STATS | go3 | 30 days
Running STATS | no2 | 1 days
Running STATS | no2 | 7 days
Running STATS | no2 | 14 days
Running STATS | no2 | 30 days
Running STATS | pm10 | 1 days
Running STATS | pm10 | 7 days
Running STATS | pm10 | 14 days
Running STATS | pm10 | 30 days
Running STATS | pm2p5 | 1 days
Running STATS | pm2p5 | 7 days
Running STATS | pm2p5 | 14 days
Running STATS | pm2p5 | 30 days


### Machine Learners

In [ ]:
for pollutant_name, pollutant_ in pollutants_dict.items():

    if pollutant_name == 'go3' or pollutant_name == 'pm10' or pollutant_name == 'no2':
        continue

    for horizon_label, experiment_ in experiments_dict.items():

        try:
            clear_output(wait=True)
            print(f"Running ML | {pollutant_name} | {horizon_label}")

            # -------------------------------
            # MODEL 
            # -------------------------------
            lgb_model = lgb.LGBMRegressor(
                n_estimators=500,
                learning_rate=0.05,
                num_leaves=31,
                subsample=0.8,
                colsample_bytree=0.8,
                random_state=SEED,
                n_jobs=1,
            )
            rf_model = RandomForestRegressor(
                n_estimators=500,
                max_depth=10,
                random_state=SEED,
                n_jobs=1,
            )
            xgb_model = xgb.XGBRegressor(
                n_estimators=500,
                learning_rate=0.05,
                max_depth=6,
                subsample=0.8,
                colsample_bytree=0.8,
                random_state=SEED,
                n_jobs=1,
            )

            # -------------------------------
            # MLForecast (lags + calendar)
            # -------------------------------
            mlf = MLForecast(
                models={
                    'RandomForest': rf_model,
                    'XGBoost': xgb_model,
                    'LightGBM': lgb_model,
                },
                freq=FREQ,
                lags=[1, 2, 4, 8, 16, 24, 56, 112],
                lag_transforms={
                    8:  [RollingMean(window_size=8), ExpandingMean()],
                    56: [RollingMean(window_size=56)],
                },
                date_features=[
                    'hour',       
                    'dayofweek',  
                    'month',      
                    'dayofyear',  
                ],
            )

            # -------------------------------
            # DATA
            # -------------------------------
            df_ = pollutant_['df'].copy()
            df_ = df_.sort_values(['unique_id', 'ds'])

            # (opcional, mas recomendado) garantir dtype datetime
            df_['ds'] = pd.to_datetime(df_['ds'])

            # -------------------------------
            # CROSS-VALIDATION
            # -------------------------------
            start_time = time.time()
            results_ml = mlf.cross_validation(
                df=df_,
                h=experiment_['horizon'],
                n_windows=experiment_['windows'],
                step_size=experiment_['step_size'],
                refit=False,
            )
            end_time = time.time()

            total_time = end_time - start_time
            results_ml['fit_time_seconds'] = total_time

            # -------------------------------
            # SAVE
            # -------------------------------
            save_results(
                results_ml,
                model_family="ml",
                pollutant=pollutant_name,
                horizon_label=horizon_label.replace(" ", "")
            )

            del mlf
            del rf_model, xgb_model, lgb_model
            
        except Exception as e:
            print(f"Failed on {pollutant_name} {horizon_label}: {e}")
            continue 

## Merge

In [ ]:
FULL_DIR = BASE_RESULTS / "full"
FULL_DIR.mkdir(parents=True, exist_ok=True)

def build_full_results(pollutant, horizon_label):
    """
    Concatena neural + stats + ml para um (pollutant, horizon)
    """
    dfs = []

    for family in ["neural", "stats", "ml", "dl"]:
        fpath = BASE_RESULTS / family / f"{pollutant}_{horizon_label}.csv"

        if fpath.exists():
            df = pd.read_csv(fpath)

            if 'fit_time_seconds' in df.columns:
                df = df.drop(columns=['fit_time_seconds'])
            
            dfs.append(df)
        else:
            print(f"⚠️ Missing: {fpath}")

    if not dfs:
        raise ValueError("No result files found.")

    df_full = dfs[0]
    for df in dfs[1:]:
        df_full = df_full.merge(
            df,
            on=["unique_id", "ds", "cutoff", "y"],
            how="inner"
        )

    return df_full

for pollutant_name in pollutants_dict.keys():
    for horizon_label in experiments_dict.keys():

        horizon_clean = horizon_label.replace(" ", "")

        print(f"Building FULL | {pollutant_name} | {horizon_clean}")

        df_full = build_full_results(
            pollutant=pollutant_name,
            horizon_label=horizon_clean
        )

        df_full.to_csv(
            FULL_DIR / f"{pollutant_name}_{horizon_clean}.csv",
            index=False
        )

Building FULL | go3 | 1days
Building FULL | go3 | 7days
Building FULL | go3 | 14days
Building FULL | go3 | 30days
Building FULL | no2 | 1days
Building FULL | no2 | 7days
Building FULL | no2 | 14days
Building FULL | no2 | 30days
Building FULL | pm10 | 1days
Building FULL | pm10 | 7days
Building FULL | pm10 | 14days
Building FULL | pm10 | 30days
Building FULL | pm2p5 | 1days
Building FULL | pm2p5 | 7days
Building FULL | pm2p5 | 14days
Building FULL | pm2p5 | 30days


## **Metrics**

In [ ]:
# ==================================================
# METRICS
# ==================================================

def mae(y, yhat):
    return np.mean(np.abs(y - yhat))


def mse(y, yhat):
    return np.mean((y - yhat) ** 2)


def rmse(y, yhat):
    return np.sqrt(mse(y, yhat))


def smape(y, yhat):
    denom = (np.abs(y) + np.abs(yhat)) / 2
    mask = denom != 0
    if mask.sum() == 0:
        return np.nan
    return np.mean(np.abs(y[mask] - yhat[mask]) / denom[mask])


def mae_conditional(y, yhat, threshold):
    mask = y >= threshold
    if mask.sum() == 0:
        return np.nan
    return np.mean(np.abs(y[mask] - yhat[mask]))


def bias_conditional(y, yhat, threshold):
    mask = y >= threshold
    if mask.sum() == 0:
        return np.nan
    return np.mean(yhat[mask] - y[mask])


def skill_score(model_err, baseline_err):
    if baseline_err == 0 or np.isnan(baseline_err):
        return np.nan
    return 1 - model_err / baseline_err


def extreme_event_metrics(y, yhat, threshold):
    y_event = y >= threshold
    yhat_event = yhat >= threshold

    tp = np.sum(y_event & yhat_event)
    fp = np.sum(~y_event & yhat_event)
    fn = np.sum(y_event & ~yhat_event)

    precision = tp / (tp + fp) if (tp + fp) > 0 else np.nan
    recall = tp / (tp + fn) if (tp + fn) > 0 else np.nan

    if precision > 0 and recall > 0:
        f1 = 2 * precision * recall / (precision + recall)
    else:
        f1 = np.nan

    return precision, recall, f1


# ==================================================
# EVALUATION LOOP (ROBUST & NO LEAKAGE)
# ==================================================

BASE = Path("Results/RQ1/full")
baseline_name = "Naive"

STEPS_PER_YEAR = 365 * 8  # dados 3h
WINDOW_P95 = STEPS_PER_YEAR

records = []

for file in BASE.glob("*.csv"):

    pollutant, horizon = file.stem.split("_", 1)
    print(f"Evaluating | {pollutant} | {horizon}")

    # --------------------------------------
    # LOAD PREDICTIONS
    # --------------------------------------
    df_pred = pd.read_csv(file)
    df_pred['ds'] = pd.to_datetime(df_pred['ds'])
    df_pred['cutoff'] = pd.to_datetime(df_pred['cutoff'])

    # --------------------------------------
    # LOAD FULL GROUND TRUTH
    # --------------------------------------
    df_true = pollutants_dict[pollutant]['df'].copy()
    df_true['ds'] = pd.to_datetime(df_true['ds'])

    # --------------------------------------
    # DEFINE MODEL COLUMNS (CLEAN)
    # --------------------------------------
    model_cols = [
        c for c in df_pred.columns
        if (
            c not in ["unique_id", "ds", "cutoff", "y"]
            and not c.startswith("fit_time")
            and df_pred[c].dtype in [np.float64, np.float32]
        )
    ]

    if baseline_name not in model_cols:
        raise ValueError(f"Baseline '{baseline_name}' not found.")

    # --------------------------------------
    # GROUP BY FOLD
    # --------------------------------------
    for (uid, cutoff), df_fold in df_pred.groupby(["unique_id", "cutoff"]):

        # ----------------------------------
        # TRAIN WINDOW (NO LEAKAGE)
        # ----------------------------------
        y_train_full = (
            df_true
            .query("unique_id == @uid and ds <= @cutoff")
            ['y']
            .values
        )

        if len(y_train_full) < WINDOW_P95:
            continue

        y_train_recent = y_train_full[-WINDOW_P95:]
        p95 = np.percentile(y_train_recent, 95)

        # ----------------------------------
        # TEST HORIZON
        # ----------------------------------
        y_full = df_fold['y'].values
        y_base_full = df_fold[baseline_name].values

        # baseline válido
        mask_base_valid = ~np.isnan(y_base_full)

        if mask_base_valid.sum() == 0:
            continue

        # ----------------------------------
        # EVALUATE EACH MODEL
        # ----------------------------------
        for model in model_cols:

            yhat_full = df_fold[model].values
            mask_valid = ~np.isnan(yhat_full)

            if mask_valid.sum() == 0:
                continue

            y_valid = y_full[mask_valid]
            yhat = yhat_full[mask_valid]
            y_base_valid = y_base_full[mask_valid]

            # métricas gerais do modelo
            mae_model = mae(y_valid, yhat)
            rmse_model = rmse(y_valid, yhat)
            smape_model = smape(y_valid, yhat)

            # métricas do baseline (ajustadas ao mesmo subconjunto)
            mae_base_model = mae(y_valid, y_base_valid)
            rmse_base_model = rmse(y_valid, y_base_valid)
            smape_base_model = smape(y_valid, y_base_valid)
            mae_base_model_p95 = mae_conditional(y_valid, y_base_valid, p95)

            # Skill Scores gerais
            skill_mae = skill_score(mae_model, mae_base_model)
            skill_rmse = skill_score(rmse_model, rmse_base_model)
            skill_smape = skill_score(smape_model, smape_base_model)

            # extremos
            mae_p95 = mae_conditional(y_valid, yhat, p95)
            skill_p95 = skill_score(mae_p95, mae_base_model_p95)
            bias_p95 = bias_conditional(y_valid, yhat, p95)

            precision, recall, f1 = extreme_event_metrics(
                y_valid, yhat, p95
            )

            records.append({
                "pollutant": pollutant,
                "horizon": horizon,
                "unique_id": uid,
                "cutoff": cutoff,
                "model": model,

                # métricas médias e seus respectivos skill scores
                "MAE": mae_model,
                "Skill_MAE": skill_mae,
                "RMSE": rmse_model,
                "Skill_RMSE": skill_rmse,
                "sMAPE": smape_model,
                "Skill_sMAPE": skill_smape,

                # extremos
                "MAE_p95": mae_p95,
                "Skill_p95": skill_p95,
                "Bias_p95": bias_p95,
                "Precision_p95": precision,
                "Recall_p95": recall,
                "F1_p95": f1,
            })

# ==================================================
# FINAL DATAFRAME
# ==================================================

metrics_df = pd.DataFrame(records)
metrics_df.to_csv(r'..\Masters 26\Results\RQ1\metrics.csv', index=False)

Evaluating | go3 | 14days
Evaluating | go3 | 1days
Evaluating | go3 | 30days
Evaluating | go3 | 7days
Evaluating | no2 | 14days
Evaluating | no2 | 1days
Evaluating | no2 | 30days
Evaluating | no2 | 7days
Evaluating | pm10 | 14days
Evaluating | pm10 | 1days
Evaluating | pm10 | 30days
Evaluating | pm10 | 7days
Evaluating | pm2p5 | 14days
Evaluating | pm2p5 | 1days
Evaluating | pm2p5 | 30days
Evaluating | pm2p5 | 7days


In [51]:
summary = (
    metrics_df
    .groupby(["model"])
    .agg({
        "MAE": "mean",
        "RMSE": "mean",
        "sMAPE": "mean",
        "MAE_p95": "mean",
        "Skill_p95": "mean",
        "F1_p95": "mean"
    })
    .sort_values("MAE")
)
summary

,MAE,RMSE,sMAPE,MAE_p95,Skill_p95,F1_p95
model,,,,,,
NHITS,5.482232e-09,7.119524e-09,0.433220,1.429769e-08,0.068600,0.527031
NBEATS-G,5.486953e-09,7.120524e-09,0.433298,1.428247e-08,0.088899,0.523155
NBEATS-I,5.502032e-09,7.150968e-09,0.436218,1.428918e-08,0.098030,0.516337
LightGBM,5.812858e-09,7.385411e-09,0.436081,1.421793e-08,0.077790,0.498773
LSTM,5.912126e-09,7.554950e-09,0.462718,1.465362e-08,-0.459748,0.540404
GRU,5.976189e-09,7.626346e-09,0.466126,1.490091e-08,-0.461215,0.551713
Informer,6.410303e-09,8.244619e-09,0.542345,1.994525e-08,-0.788818,0.556775
RandomForest,6.692651e-09,8.406920e-09,0.578508,1.775421e-08,-1.075025,0.681198
Theta,7.318015e-09,8.966578e-09,0.537794,1.528884e-08,0.053667,0.520314
